<a href="https://colab.research.google.com/github/sarikasea/Mythbusters/blob/main/V4_Gemini_Mythbusting_Crew_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q crewai crewai_tools litellm python-dotenv google-colab


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.1/650.1 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.9 MB/s et

In [2]:
# -*- coding: utf-8 -*-
"""V4 Mythbusting CrewAI Agent Article Writer - LiteLLM Gemini Edition"""

import os
from google.colab import userdata
import warnings
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
from IPython.display import display, Markdown
from litellm import completion



/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [3]:
# --- Set API Keys securely from Colab secrets or elsewhere as needed ---
os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

In [4]:
# Gemini model name as required by LiteLLM
GEMINI_MODEL = "gemini/gemini-1.5-pro-latest"

# Search Tool setup
search_tool = SerperDevTool()

In [5]:
from langchain_openai import ChatOpenAI

# This is the correct way to configure the LLM for your agents.
# It creates an object that CrewAI can correctly use.
llm = ChatOpenAI(
    model="gemini/gemini-1.5-pro-latest",
    api_key=os.environ.get("GEMINI_API_KEY"),
    temperature=0.7,
    verbose=True
)

print("✅ LLM object configured to use 'gemini/gemini-1.5-pro-latest'.")


✅ LLM object configured to use 'gemini/gemini-1.5-pro-latest'.


#Agents Configuration

In [6]:
# -- AGENTS CONFIGURATION --

ResearchAgent = Agent(
    name="HistoricalFactFinder",
    role=(
        "You are a specialized historical researcher. Your task is to gather detailed, "
        "fact-checked information on a given historical figure ({topic}). You must find "
        "information to cover the following specific sections: "
        "1. Era/Country, 2. Love Life, 3. Scientific/Political Contribution, "
        "4. Misunderstanding/Scandal, 5. End of Life/Legacy, "
        "6. Recommended books about them, 7. Notable Quotes from them, "
        "8. Songs/Movies about them."
    ),
    goal=(
        "Compile a comprehensive report on {topic} with detailed notes and verifiable "
        "sources for each of the eight required sections. This report will be used to "
        "write a structured, myth-busting article."
    ),
    backstory=(
        "You are a meticulous historian with a passion for primary sources and "
        "academic databases. You excel at separating fact from fiction and providing "
        "the raw material for compelling historical narratives."
    ),
    tools=[search_tool],
    llm=ask_gemini,
    memory=True,
    allow_delegation=False,
    verbose=True,
)

WriterAgent = Agent(
    name="NarrativeWeaver",
    role=(
        "You are an expert writer specializing in historical biographies. Your task is to "
        "transform research notes about {topic} into an engaging myth-busting article."
    ),
    goal=(
        "Write a captivating article about {topic} that is clearly structured into the "
        "following eight sections: 1. Historical Context (Era/Country), "
        "2. Personal Life & Relationships, 3. Major Contributions & Achievements, "
        "4. Scandals & Misconceptions, 5. Final Years & Lasting Legacy, "
        "6. Further Reading (Recommended Books), 7. Memorable Quotes, "
        "8. Pop Culture Portrayals (Songs/Movies). The article must be engaging, "
        "educational, and based strictly on the provided research."
    ),
    backstory=(
        "As a journalist and cultural analyst for a popular history blog, you know how to "
        "make history come alive. You craft narratives that are not only informative but "
        "also challenge common misinformation, especially concerning women in history."
    ),
    llm=ask_gemini,
    memory=True,
    allow_delegation=False,
    verbose=True,
)

EditorAgent = Agent(
    name="ContentChief",
    role=(
        "You are a meticulous editor with expertise in feminist history and public education."
        "You will review a draft article for clarity, factual accuracy, and tone."
    ),
    goal=(
        "Polish the myth-busting article about {topic}, ensuring it aligns with the goal "
        "of accessible public education. You must verify that: "
        "1. All facts are clearly explained and well-cited. "
        "2. The tone is professional yet accessible. "
        "3. All eight required sections (Era, Love Life, Contributions, Scandal, Legacy, "
        "Books, Quotes, Media) are present, well-developed, and seamlessly integrated."
    ),
    backstory=(
        "With years of experience editing for historical journals and online magazines, "
        "you have a sharp eye for detail and a deep understanding of how to present "
        "complex historical truths to a broad audience. Your standard is excellence."
    ),
    llm=ask_gemini,
    memory=True,
    allow_delegation=False,
    verbose=True,
)

NameError: name 'ask_gemini' is not defined

# -- TASKS CONFIGURATION & WORKFLOW --

In [ ]:
def run_myth_busting_workflow(topic):
    research_task = Task(
        agent=ResearchAgent,
        description=(
            f"Find detailed, fact-checked information on the historical figure: {topic}. "
            "Your research must provide material for all eight of the required sections. "
            "Focus on uncovering details that can bust common myths or add surprising depth."
        ),
        expected_output=(
            "A comprehensive report as a single text document. The report must be clearly "
            "structured with headings for each of the 8 required sections, containing detailed "
            "notes and verifiable sources for each section."
        ),
    )
    writing_task = Task(
        agent=WriterAgent,
        description=(
            f"Using the provided research report on {topic}, write a captivating, myth-busting "
            "article of approximately 1200 words. The article's structure must strictly "
            "follow the 8 required sections. Weave the research into a compelling narrative "
            "that is both educational and engaging."
        ),
        expected_output=(
            "A complete, well-written article draft, formatted with clear headings for each of the "
            "8 sections. The tone should be engaging and accessible to a general audience."
        ),
        context=[research_task]
    )
    editing_task = Task(
        agent=EditorAgent,
        description=(
            f"Review the provided article draft about {topic}. Your review must focus on several key areas: "
            "1. Verify that all 8 required sections are present, well-developed, and logically structured. "
            "2. Check all facts against the source material for accuracy. "
            "3. Polish the writing for clarity, engagement, and consistent tone. "
            "4. Ensure the article successfully fulfills its 'myth-busting' goal."
        ),
        expected_output=(
            "The final, publishable version of the article on {topic}. It should be free of errors, "
            "factually pristine, and ready for an audience."
        ),
        context=[writing_task]
    )

    crew = Crew(
        agents=[ResearchAgent, WriterAgent, EditorAgent],
        tasks=[research_task, writing_task, editing_task],
        verbose=True,
    )
    result = crew.kickoff(inputs={"topic": topic})
    display(Markdown(f"## Final Article for **{topic}**\n---\n{result}"))



# List of topics to process

In [ ]:
topics_to_process = ["Cleopatra", "Marie Curie"]

for topic in topics_to_process:
    print(f"\n\n=============================================")
    print(f"🚀 Starting New Workflow for Topic: {topic}")
    print(f"=============================================")
    run_myth_busting_workflow(topic)

In [ ]:
from litellm import completion
import os

assert os.getenv("GEMINI_API_KEY") is not None, "GEMINI_API_KEY not set!"

resp = completion(
    model="gemini/gemini-1.5-pro-latest",
    messages=[{"role": "system", "content": "You are a helpful assistant."},
              {"role": "user", "content": "Say hello!"}],
)
print(resp.choices[0].message.content)